# Capstone Unified Bayesian Optimizer
Use this notebook to optimize any of the 8 black-box functions using Bayesian Optimization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from pathlib import Path

In [ ]:
# Select function number (1 to 8)
function_number = 1  # Change to 2 for Function 2, ..., up to 8
data_dir = Path(".")

In [ ]:
# Resolve filenames
if function_number == 8:
    X = np.load(data_dir / "initial_inputs.npy")
    Y = np.load(data_dir / "initial_outputs.npy")
else:
    X = np.load(data_dir / f"initial_inputs_fn{function_number}.npy")
    Y = np.load(data_dir / f"initial_outputs_fn{function_number}.npy")

In [ ]:
# Optional: Add past queries if they match input dimension
try:
    import ast
    with open("queries.txt") as f:
        queries = ast.literal_eval(f.read())
    with open("observations.txt") as f:
        obs_lines = f.readlines()
        observations = [ast.literal_eval(line.strip()) for line in obs_lines]

    queries = [np.array(q).flatten() for q in queries]
    observations = [np.array(y) for y in observations]

    # Select function-specific observations
    queried_X = []
    queried_Y = []
    for q, y in zip(queries, observations):
        if len(q) == X.shape[1]:  # Match input dimension
            queried_X.append(q)
            queried_Y.append(y[function_number - 1])

    if queried_X:
        X = np.vstack([X, queried_X])
        Y = np.concatenate([Y, queried_Y])
        print(f"✔ Added {len(queried_X)} queried points to initial data.")
except Exception as e:
    print("⚠ Could not load additional queries:", e)

In [ ]:
# Fit Gaussian Process
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, normalize_y=True)
gp.fit(X, Y)

# Random candidates
np.random.seed(42)
X_candidates = np.random.uniform(0, 1, size=(1000, X.shape[1]))

# Predict and compute Expected Improvement
mu, sigma = gp.predict(X_candidates, return_std=True)
y_max = np.max(Y)
Z = (mu - y_max) / sigma
ei = (mu - y_max) * norm.cdf(Z) + sigma * norm.pdf(Z)
ei[sigma == 0.0] = 0

# Suggest next X
next_idx = np.argmax(ei)
next_X = X_candidates[next_idx]

print(f"🎯 Suggested next X to query for Function {function_number}:")
print(next_X)

# Plot EI
plt.figure(figsize=(10, 4))
plt.plot(ei, label='Expected Improvement')
plt.title(f'Expected Improvement (Function {function_number})')
plt.xlabel('Candidate Index')
plt.ylabel('EI')
plt.legend()
plt.show()